In [1]:
from credmark.cmf.ipython import create_cmf
from credmark.cmf.types import Token, Contract, Address, Account, BlockNumber, Records, JoinType
import requests
import pandas as pd
from dotenv import load_dotenv
import os

cmf_param = {
    'chain_id': 1,
}

os.chdir('/home/thomashepner/credit-cooperative/credmark-models-py/')
context, _model_loader = create_cmf(cmf_param)

ModelRunRequestError: Model ledger.block-number run request response missing both output and error

In [ ]:
rain_mainnet_factory = Contract('0x389c6f04D39fF038cC99F94BcE338D2f5Eb64a67')
rain_factory_proxy = Contract('0x31EBf70312f488D0bdAc374b340f0D01dBf153B5')

rain_controller = Contract('0xE5D3d7da4b24bc9D2FDA0e206680CD8A00C0FeBD')
usdc_addr = Token("USDC")

load_dotenv()

RAIN_DEVELOPER_API_KEY = os.environ.get('RAIN_DEVELOPER_API_KEY')
RAIN_CARD_SPEND_ENDPOINT = os.environ.get('RAIN_CARD_SPEND_ENDPOINT')


start_date = BlockNumber.from_ymd(2022, 10, 1)
current_date=context.block_number


In [ ]:
# rain_mainnet_factory.ledger.events.NewRainCollateral
rain_factory_proxy.ledger.events.NewRainCollateral

### Get Rain Collateral Contracts

In [ ]:
with rain_factory_proxy.ledger.events.NewRainCollateral as q:
    rain_collateral_contracts_df = q.select(columns=[q.EVT__COLLATERALPROXY],
                    order_by=q.BLOCK_NUMBER.desc(),
                    limit=1000).to_dataframe()

rain_collateral_contracts_df.head(n=10)


### Get Transaction History for each Rain Collateral Contract

In [ ]:
# create new dataframe to store the historical transfers of USDC to and from the collateral proxy
transfers_df = pd.DataFrame(columns=['block_number', 'to_address', 'from_address', 'token_address', 'transaction_hash', 'value', 'contract_address'])

# iterate through each Rain collateral contract
for index, row in rain_collateral_contracts_df.iterrows():
    # print(row['evt__collateralProxy'])
    collateral_proxy = Address(row['evt__collateralProxy'])
    print('Collateral Proxy: ' + collateral_proxy)
    print('Num Rows: ', transfers_df.shape[0])

    # for each row, get historical transfers of USDC to and from the collateral proxy
    with context.ledger.TokenTransfer as q:
        transfers = q.select(
            columns=[q.BLOCK_NUMBER,
                    q.TO_ADDRESS,
                    q.FROM_ADDRESS,
                    q.TOKEN_ADDRESS,
                    q.TRANSACTION_HASH],
            aggregates=[((f'CASE WHEN {q.TO_ADDRESS.eq(collateral_proxy)} '
                      f'THEN {q.VALUE} ELSE {q.VALUE.neg_()} END'),
                    'value'),
                    ((f'CASE WHEN {q.TO_ADDRESS.eq(collateral_proxy)} '
                      f'THEN {q.TO_ADDRESS} ELSE {q.FROM_ADDRESS} END'),
                    'contract_address'),
                    ],
            where=q.TOKEN_ADDRESS.eq('0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48').and_(q.TO_ADDRESS.eq(collateral_proxy)
                            .or_(q.FROM_ADDRESS.eq(collateral_proxy)).parentheses_())).to_dataframe()

        # concatenate the transfers to the transfers dataframe
        # if (transfers.shape[0] > 0):
        print(transfers_df.columns)
        print(transfers.columns)
            # transfers_df = pd.concat([transfers_df, transfers])


# # Reorder the columns
# transfers_df = transfers_df[['block_number', 'contract_address', 'to_address', 'from_address', 'token_address', 'transaction_hash', 'value']]

# transfers_df.head(n=10)


In [ ]:
collateral_proxy = Address('0x8FB0d9b49B8493cF77c6AE70d40d34A81078F29a')

with context.ledger.TokenTransfer as q:
    transfers = q.select(
        columns=[q.BLOCK_NUMBER,
                    q.TO_ADDRESS,
                    q.FROM_ADDRESS,
                    q.TOKEN_ADDRESS,
                    q.TRANSACTION_HASH],
        aggregates=[((f'CASE WHEN {q.TO_ADDRESS.eq(collateral_proxy)} '
                      f'THEN {q.VALUE} ELSE {q.VALUE.neg_()} END'),
                    'value'),
                    ((f'CASE WHEN {q.TO_ADDRESS.eq(collateral_proxy)} '
                      f'THEN {q.TO_ADDRESS} ELSE {q.FROM_ADDRESS} END'),
                    'contract_address'),
                    ],
        where=q.TOKEN_ADDRESS.eq('0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48').and_(q.TO_ADDRESS.eq(collateral_proxy)
                            .or_(q.FROM_ADDRESS.eq(collateral_proxy)).parentheses_())).to_dataframe()

transfers.head(n=20)

In [ ]:
with rain_factory_proxy.ledger.events.NewRainCollateral as q:
    res = q.select(columns=[q.EVT__COLLATERALPROXY],
                    order_by=q.BLOCK_NUMBER.desc(),
                    limit=1000).to_dataframe()

print(res)

# iterate through each row in res

# for each row, get the balance of the collateral proxy

# for index, row in res.iterrows():
#     # print(row['evt__collateralProxy'])
#     collateral_proxy = Address(row['evt__collateralProxy'])
#     balance = usdc_addr.functions.balanceOf(collateral_proxy).call()
#     print(balance)
#     # with collateral_proxy.balance_of(usdc_addr) as q:
#     #     res = q.select(columns=[q.EVT__VALUE],
#     #                 order_by=q.BLOCK_NUMBER.desc(),
#     #                 limit=200).to_dataframe()
#     # print(res)
#     # # iterate through each row in res
#     # # for each row, get the balance of the collateral proxy
#     # for index, row in res.iterrows():
#     #     print(row['VALUE'])
#     #     # if the balance is greater than 0, then we need to call the rain card spend endpoint
#     #     if row['VALUE'] > 0:
#     #         print('calling rain card spend endpoint')
#     #         # call the rain card spend endpoint



In [ ]:


transfers_df.loc[''].head(n=20)

In [ ]:
rain_collateral_contract = Address('0x8FB0d9b49B8493cF77c6AE70d40d34A81078F29a')

In [ ]:
print(context.ledger.tables())
print(context.ledger.table("TokenTransfer").columns)

In [ ]:
with context.ledger.TokenTransfer as q:
    transfers = q.select(
        columns=[q.BLOCK_NUMBER,
                 #  BlockNumber(q.BLOCK_NUMBER).timestamp_datetime.strftime("%Y-%M-%d"),
                 q.VALUE,
                 q.TO_ADDRESS,
                 q.FROM_ADDRESS,
                 q.TOKEN_ADDRESS,
                 q.TRANSACTION_HASH],
        aggregates=[((f'CASE WHEN {q.TO_ADDRESS.eq(rain_collateral_contract)} '
                      f'THEN {q.VALUE} ELSE {q.VALUE.neg_()} END'),
                    'value'),
                    ],
        where=q.TOKEN_ADDRESS.eq('0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48').and_(q.TO_ADDRESS.eq(rain_collateral_contract)
                          .or_(q.FROM_ADDRESS.eq(rain_collateral_contract)).parentheses_())).to_dataframe()


transfers.head(n=20)

In [ ]:
# Make cumulative dataframe
transfers_w_cumulative = transfers.copy()
transfers_w_cumulative['cumulative'] = transfers['value'].cumsum()
transfers_w_cumulative['value_usdc'] = transfers['value'] / 10**6
transfers_w_cumulative['cumulative_usdc'] = transfers_w_cumulative['cumulative'] / 10**6

transfers_w_cumulative.head(n=20)

In [ ]:
rain_controller.ledger

In [ ]:
rain_controller.ledger.events.Liquidation

In [ ]:
with rain_controller.ledger.events.Liquidation as q:
    liquidations = (q.select(
        columns=[
            q.EVT__COLLATERALPROXY,
            q.EVT__ASSETS,
            q.EVT__AMOUNTS
        ],
        where=q.BLOCK_NUMBER.between_(start_date, current_date)
    )).to_dataframe()

display(liquidations)

In [ ]:
rain_controller.ledger.events.Payment

In [ ]:
with rain_controller.ledger.events.Payment as q:
    payments = (q.select(
        columns=[
            q.EVT__COLLATERALPROXY,
            q.EVT__ASSETS,
            q.EVT__AMOUNTS
        ],
        where=q.BLOCK_NUMBER.between_(start_date, current_date)
    )).to_dataframe()

display(payments)

In [ ]:
rain_controller.ledger.events.Withdrawal

In [ ]:
with rain_controller.ledger.events.Withdrawal as q:
    withdrawals = (q.select(
        columns=[
            q.EVT__COLLATERALPROXY,
            q.EVT__ASSET,
            q.EVT__AMOUNT
        ],
        where=q.BLOCK_NUMBER.between_(start_date, current_date)
    )).to_dataframe()

display(withdrawals)

In [ ]:
def get_card_spend_dataframe():
    # Headers (if required)
    headers = {
        'API-Key': RAIN_DEVELOPER_API_KEY,
        'accept': 'application/json'
    }

    # Make the GET request
    response = requests.get(RAIN_CARD_SPEND_ENDPOINT, headers=headers)

    # Check if the request was successful
    if response.status_code == 200:
        # Process the response if successful
        data = response.json()
        print(data)
    else:
        # Handle errors (you can print or log the error response)
        print(f"Error: {response.status_code}")

    # create dataframe
    df = pd.DataFrame([data])

    # add date column
    df.insert(0, 'date', current_date)


    # drop dueBalance
    df = df.drop(columns=['dueBalance'])

    # add totalBalance
    df['totalBalance'] = df['pendingBalance'] + df['postedBalance']

    return df

In [ ]:
latest_card_spend_df = get_card_spend_dataframe()